In [ ]:
import import_ipynb
import numpy as np
from scipy.sparse import lil_matrix
from fem_processing import master_domain, gaussian_quadrature

# `jacobian()`

In [ ]:
def jacobian(mesh_data, cell):

    # Get the global coordinates of the nodes
    xi = [mesh_data['nodes'][node]['xg'][0] for node in cell['conn_sorted']]
    yi = [mesh_data['nodes'][node]['xg'][1] for node in cell['conn_sorted']]

    # Calcular o Jacobiano
    Je = np.array([
        [xi[1] - xi[0], xi[2] - xi[0]], 
        [yi[1] - yi[0], yi[2] - yi[0]]
    ])

    return Je

# `local_matrices()`

In [ ]:
def local_matrices(FINITE_ELEMENT, mesh_data, cell):
    # Initialize the local matrices
    Ne = len(cell['conn_edge'])
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    fe = np.zeros((Ne, 1))      # Vetor de carga do elemento

    # Get the Gauss points and weights
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

    # Cálculo da matriz Jacobiana
    Je = jacobian(mesh_data, cell)

    # Determinante e inversa transposta do Jacobiano
    Jdet, JinvT = np.abs(np.linalg.det(Je)), np.linalg.inv(Je).T       

    # Matriz de rigidez do elemento
    Se = 2 / Jdet * np.array([
        [1, -1, 1], 
        [-1, 1, -1],
        [1, -1, 1]
    ])

    for ik, (xik, wk) in enumerate(zip(gauss_points, gauss_weights)):
        # Material properties
        ma = cell['mass_term'][ik]
        rho = cell['source'][ik]

        # Função de forma vetorial no ponto de integração
        phi_hat = master_domain.shape_functions_n0(*xik)

        # Montagem da matriz de massa
        for i in range(Ne):
            # Transformação de Piola Covariante
            phi_i = JinvT @ phi_hat[i]

            # Vetor de cargas
            fe[i] += (rho.T @ phi_i).item() * Jdet * wk
            
            for j in range(Ne):
                # Transformação de Piola Covariante
                phi_j = JinvT @ phi_hat[j]
                
                # Matriz de massa
                Me[i, j] += ma * (phi_i.T @ phi_j).item() * Jdet * wk 
    
    return Se, fe, Me

# `reduced_global_matrices()`

In [ ]:
def reduced_global_matrices(FINITE_ELEMENT, mesh_data, jw=0):
    # Arestas livres
    free_edges = {key: value
                   for key, value in mesh_data['edges'].items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_edge: idx
                          for idx, global_edge in enumerate(free_edges.keys())}

    # Initialize the global matrix and vector
    Sgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
    Mgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
    fgr = lil_matrix((len(free_edges), 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values():
        # Compute the local data
        Se, fe, Me = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        for i, ig in enumerate(cell['conn_edge']):      # Loop sobre os índices globais do elemento
            if ig in free_edges:                        # Verifica se o nó 'i' é livre
                ig_red = global_to_reduced[ig]          # Índice reduzido
                
                for j, jg in enumerate(cell['conn_edge']):
                    if jg in free_edges:
                        jg_red = global_to_reduced[jg]
                        Sgr[ig_red, jg_red] += Se[i, j]
                        Mgr[ig_red, jg_red] += Me[i, j]

                    # A aresta global é de Dirichlet
                    else:
                        # Contribuição de Dirichlet para o vetor reduzido
                        fgr[ig_red, 0] += -(Se[i, j] + jw * Me[i, j]) * mesh_data['edges'][jg]['bc']['value']
                
                # preenche o vetor global de carga
                fgr[ig_red, 0] += fe[i]

    return Sgr, fgr, Mgr

# `global_potentials()`

In [ ]:
def global_potentials(mesh_data, free_potentials):
    """
    Monta o vetor de potenciais globais V combinando potenciais calculados (livres) e prescritos (Dirichlet),
    utilizando o mapeamento global_to_reduced.

    Parâmetros:
    - mesh_data: Dados da malha, contendo nós e condições de contorno.
    - free_potentials: Potenciais calculados para os graus de liberdade livres (nós não-Dirichlet).

    Retorna:
    - V: Vetor contendo os potenciais globais de todos os nós.
    """
    # Inicializa o vetor de potenciais globais com zeros
    uh = {}

    # Arestas livres
    free_edges = {key: value
                   for key, value in mesh_data['edges'].items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_edge: idx
                         for idx, global_edge in enumerate(free_edges.keys())}

    # Preenche o vetor solução
    for key, edge in mesh_data['edges'].items():
        # Verifica se a aresta é livre
        if key in free_edges:
            # Mapeamento global para reduzido
            uh[int(key)] = free_potentials[global_to_reduced[key]]

        # Nó de Dirichlet
        else:
            uh[int(key)] = edge['bc']['value']

    return uh

# `apply_simple_dirichlet()`

In [ ]:
def apply_simple_dirichlet(Sg, fg, mesh_data):
    for key, edge in mesh_data['edges'].items():
        # Verifica se a aresta é de Dirichlet
        if edge['bc']['type'] == 'Dirichlet':
            e = int(key) - 1
            
            # Zera a linha e coluna correspondente a aresta de Dirichlet
            Sg[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Sg[e, e] = 1
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            fg[e] = edge['bc']['value']

    return Sg, fg

# `global_matrices()`

In [ ]:
def global_matrices(FINITE_ELEMENT, mesh_data):
    Nedges = len(mesh_data['edges'])

    # Inicializa a matriz global como uma matriz esparsa zero (tamanho NxN)
    Sg = lil_matrix((Nedges, Nedges), dtype='complex128')
    Mg = lil_matrix((Nedges, Nedges), dtype='complex128')
    fg = lil_matrix((Nedges, 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values(): 
        # Computa as matrizes locais
        Se, fe, Me = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        # loop sobre os nós locais de cada elemento
        for i, ig in enumerate(cell['conn_edge']):
            ig = int(ig)-1
            for j, jg in enumerate(cell['conn_edge']):
                jg = int(jg)-1
                Sg[ig, jg] += Se[i, j]
                Mg[ig, jg] += Me[i, j]
                
            # preenche o vetor global b
            fg[ig, 0] += fe[i]

    return Sg, fg, Mg

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil, 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.